# Advanced Object-Oriented Programming: Metaclasses and Class Decorators

A `class` is a "factory for objects (instances)"

What is a "factory for classes?"

In [1]:
class MyClass1:
    pass

In [2]:
obj = MyClass1()
type(obj)

__main__.MyClass1

In [3]:
type(MyClass1)

type

In [4]:
%%python2
def meta(name, bases, dct):
    print("Calling the metaclass")
    return type(name, bases, dct)

class MyClass2(object):
    __metaclass__ = meta


Calling the metaclass


In [6]:
def meta(name, bases, dct):
    print(name, bases, dct)
    return type(name, bases, dct)

class MyClass2(metaclass=meta):
    a = 'foo'
    def __init__(self):
        pass


MyClass2 () {'__module__': '__main__', '__qualname__': 'MyClass2', 'a': 'foo', '__init__': <function MyClass2.__init__ at 0x103e1f680>}


In [7]:
dct = {'a': 'foo'}
MyClass2 = meta('MyClass2', (), dct)

MyClass2 () {'a': 'foo'}


In [8]:
type(MyClass2)

type

In [9]:
class WithAMeta(type):
    def __new__(meta, name, bases, dct):
        dct['a'] = 5
        return super().__new__(meta, name, bases, dct)
    
class MyClass3(metaclass=WithAMeta):
    pass

In [10]:
MyClass3.a

5

In [11]:
MyClass3 = WithAMeta('MyClass3', (object,), {})

In [12]:
type(MyClass3)

__main__.WithAMeta

In [13]:
MyClass3.a

5

### Metaclass trick: instance level property declaration

In [35]:
from copy import deepcopy

def property_meta(name, bases, dct):
    public_attributes = {
        key: value 
        for key, value in dct.items()
        if not callable(value)
        if not key.startswith('_')
    }
    dct['_public_attributes'] = public_attributes
    return type(name, bases, dct)

class PublicClass(metaclass=property_meta):
    a = 5
    b = 6
    c = [7, 8, 9]
    
    def __init__(self):
        for key, value in self._public_attributes.items():
            setattr(self, key, deepcopy(value))

In [36]:
PublicClass._public_attributes

{'a': 5, 'b': 6, 'c': [7, 8, 9]}

In [37]:
pc = PublicClass()

In [38]:
pc.__dict__

{'a': 5, 'b': 6, 'c': [7, 8, 9]}

In [39]:
pc.c.append(10)

In [40]:
pc.c

[7, 8, 9, 10]

In [41]:
PublicClass.c

[7, 8, 9]

(a better way is to use dataclasses from the standard lib)

### Metaclass use case: collecting properties ('declarative programming')

In [42]:
class ImportantField:
    def __init__(self, value, name=None):
        self.value = value
        self.name = name
        
class CollectorMeta(type):
    def __new__(meta, name, bases, dct):
        important = []
        for k, v in dct.items():
            if isinstance(v, ImportantField):
                if v.name is None:
                    v.name = k
                important.append(v)
        dct['_important'] = important
        return type.__new__(meta, name, bases, dct)
        
class MyBase(metaclass=CollectorMeta):
    """Use regular inheritance to get the metaclass 'for free'"""

    def __repr__(self):
        l = ['<{}'.format(self.__class__.__name__)]
        for fld in self._important:
            l.append(' {}={}'.format(fld.name, fld.value))
        l.append('>')
        return ''.join(l)
    

In [43]:
# Here is the "application-level" code you'd write
    
class MyClass4(MyBase):
    a = ImportantField(1)
    b = ImportantField(2)
    c = ImportantField(3)

In [44]:
MyClass4()

<MyClass4 a=1 b=2 c=3>

In [46]:
MyClass4._important

### Aside: metaclass inheritance

If ClassSub with metaclass MetaClassB extends ClassSuper with MetaClassA, then
MetaClassB must be a subclass of MetaClassA:

In [50]:
class MetaClassA(type):
    pass

class MetaClassB(MetaClassA):
    pass

class ClassSuper(metaclass=MetaClassA):
    pass

class ClassSub(ClassSuper, metaclass=MetaClassB):
    pass

### Metaclass use case: class registries

In [59]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = type.__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def lookup_meta(self, name):
        return self._registry[name]

class RegistryBase(metaclass=RegistryMeta):
    
    @classmethod
    def lookup(cls, name):
        return cls._registry[name]

In [60]:
class Registered1(RegistryBase):
    a = ImportantField('Registered2')

class Registered2(RegistryBase):
    pass
    
class Registered3(object, metaclass=RegistryMeta):
    pass

In [61]:
RegistryBase.lookup('Registered1')

__main__.Registered1

In [62]:
RegistryBase.lookup('Registered2')

__main__.Registered2

In [63]:
RegistryBase.lookup_meta('Registered1')

__main__.Registered1

In [64]:
RegistryBase.lookup_meta('Registered2')

__main__.Registered2

### Metaclass trick: class operators

In [73]:
class RegistryMeta(type):
    _registry = {}
    
    def __new__(meta, name, bases, dct):
        cls = type.__new__(meta, name, bases, dct)
        meta._registry[name] = cls
        return cls
    
    def __getitem__(cls, name):
        return cls._registry[name]
    
class RegistryBase(object, metaclass=RegistryMeta):
    name = 'base'
  
#     sadly, this does not work...
#     @classmethod
#     def __getitem__(cls, name):
#         return cls._registry[name]
    
class Registered1(RegistryBase):
    pass

class Registered2(RegistryBase):
    pass

In [74]:
RegistryBase['Registered1']

__main__.Registered1

In [75]:
RegistryBase['Registered2']

__main__.Registered2

Trick: inheritance via `+`

In [79]:
class HierMeta(type):
    
    def __add__(cls, other):
        return type(
            f'<AnonCls>({cls.__name__}, {other.__name__})', 
            (cls, other), 
            {}
        )

class HierBase(metaclass=HierMeta): pass

class R1(HierBase): pass
class R2(HierBase): pass

R3 = R1 + R2


In [80]:
R3

__main__.<AnonCls>(R1, R2)

In [81]:
R3.mro()

[__main__.<AnonCls>(R1, R2),
 __main__.R1,
 __main__.R2,
 __main__.HierBase,
 object]

### Class decorators: cheap substitute for metaclasses

Recall that

```python
@foo
def func(...):
    ...
```

really means

```python
def func(...):
    ...
func = foo(func)
```

We can do the same with classes:

```python
@foo
class Bar:
    ...
```

means

```python
class Bar:
    ...
Bar = foo(Bar)
```

In [91]:
class Registry(object):

    def __init__(self):
        self._registry = {}

    def register(self, cls):
        self._registry[cls.__name__] = cls
        return cls

    def __getitem__(self, name):
        return self._registry[name]
    
r = Registry()

In [90]:
# "Application" code
@r.register
class Registered1:
    pass

@r.register
class Registered2():
    pass

In [87]:
r._registry

{'Registered1': __main__.Registered1, 'Registered2': __main__.Registered2}

In [88]:
r['Registered1']

__main__.Registered1

In [89]:
r['Registered2']

__main__.Registered2

Real-world example: [Flask-RESTPlus][rest+]

[rest+]: https://flask-restplus.readthedocs.io/en/stable/quickstart.html#a-minimal-api

## Metaclass semantics

bar = Foo()

- if Foo is a class, bar is an instance
- if Foo is a metaclass, bar is a class

- `type` is a metaclass
- `object` is class
- `'foo'` is an instance (of the class `str`, which has metaclass `type`)

# Descriptors

... or "ever wonder how `@property`, `@classmethod`, and `@staticmethod` work?"

**Descriptors** are object which contain one or more of the following magic methods, and which occur in a class body:

- `__get__(self, object, type)` - called when the descriptor attribute is looked up (e.g. `getattr()`)
- `__set__(self, object, value)` - called when the descriptor attribute is set (e.g. `setattr()`)
- `__delete__(self, object)` - called when the descriptor attribute is deleted (e.g. `delattr()` or `del obj.attr`)

In [115]:
class MyDesc:
    
    def __get__(self, obj, typ):
        print(f'Calling __get__({self}, {obj}, {typ})')
        if obj is None:
            return self

    def __set__(self, obj, value):
        print(f'Calling __set__({self}, {obj}, {value})')

    def __delete__(self, obj):
        print(f'Calling __del__({self}, {obj})')

        
class MyClass:
    a = MyDesc()
    
    def __repr__(self):
        return '<Instance of MyClass>'


In [118]:
myobj = MyClass()

In [119]:
MyClass.a

Calling __get__(<__main__.MyDesc object at 0x103f41290>, None, <class '__main__.MyClass'>)


In [120]:
myobj.a

Calling __get__(<__main__.MyDesc object at 0x103f41290>, <Instance of MyClass>, <class '__main__.MyClass'>)


In [121]:
# MyClass.a = 5  # Overwrites the descriptor, so don't do this

In [122]:
myobj.a = 20

Calling __set__(<__main__.MyDesc object at 0x103f41290>, <Instance of MyClass>, 20)


In [123]:
del myobj.a

Calling __del__(<__main__.MyDesc object at 0x103f41290>, <Instance of MyClass>)


Let's re-implement @property:

In [129]:
class myproperty():

    def __init__(self, getter, setter=None, deleter=None):
        self._getter = getter
        self._setter = setter
        self._deleter = deleter
        
    def __get__(self, obj, typ):
        if obj is None:
            return self
        return self._getter(obj)
    
    def __set__(self, obj, value):
        if self._setter is None:
            raise TypeError('value is read-only')
        self._setter(obj, value)
        
    def __delete__(self, obj):
        if self._deleter is None:
            raise TypeError('value is undeleteable')
        self._deleter(obj)
        
    def setter(self, setter):
        """Descorator to add a setter"""
        self._setter = setter
        return self
        
            

In [130]:
class Foo():
    
    @myproperty
    def bar(self):
        print('Calling getter for bar')
        return 'barval'
    
    @bar.setter
    def bar(self, value):
        print('Calling setter for bar')
        
foo = Foo()

In [131]:
foo.bar

Calling getter for bar


'barval'

In [132]:
foo.bar = 10

Calling setter for bar


In [133]:
del foo.bar

TypeError: value is undeleteable

In [134]:
Foo.bar

## Descriptor types

- A **data descriptor** is a descriptor that defines both `__get__` and `__set__`
- A **non-data descriptor** is a descriptor that defines only `__get__`

> Data and non-data descriptors differ in how overrides are calculated with respect to entries in an instance’s dictionary. If an instance’s dictionary has an entry with the same name as a data descriptor, the data descriptor takes precedence. If an instance’s dictionary has an entry with the same name as a non-data descriptor, the dictionary entry takes precedence.

So order of precedence in attribute access is:

- data descriptor
- instance `__dict__`
- non-data descriptor

In [135]:
class Foo(): 
    pass

foo = Foo()
foo.a = 5

In [136]:
foo.__dict__

{'a': 5}

In [137]:
class DataDescriptor():
    
    def __get__(self, object, type):
        if object is None:
            return self
        return 'data descriptor value'
    
    def __set__(self, object, value):
        # Just make it a read-only data descriptor
        raise TypeError('read-only property')

In [138]:
class NonDataDescriptor():
    
    def __get__(self, object, type):
        if object is None:
            return self
        return 'non-data descriptor value'
        

In [139]:
class MyClass():
    data = DataDescriptor()
    nondata = NonDataDescriptor()

In [140]:
obj = MyClass()
obj.__dict__.update(
    data='instance data',
    nondata='instance nondata'
)

Data descriptors have precedence over instance data:

In [141]:
obj.data

'data descriptor value'

Instance data has precendence over non-data descriptors:

In [142]:
obj.nondata

'instance nondata'

## Descriptor use case: cached property

In [143]:
class cached_property:
    
    def __init__(self, getter):
        self._getter = getter
        self._name = getter.__name__
    
    def __get__(self, obj, typ):
        if obj is None:
            return self
        value = self._getter(obj)
        setattr(obj, self._name, value) # put the value in the instance __dict__
        return value

In [144]:
class CachedExample:
    
    @cached_property
    def prop(self):
        print('Calculating CachedExample.prop')
        return 42

In [150]:
ce = CachedExample()

In [151]:
ce.prop # adds prop to ce.__dict__

Calculating CachedExample.prop


42

In [152]:
ce.prop  # Since prop is non-data descriptor, it is not even accessed here

42

In "real life", this is implemented in `functools.cached_property` (in Python 3.8+)

Open [Advanced OOP Lab](./advanced-oop-lab.ipynb)